In [1]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [2]:
data = json.load(open('dataset.json','r'))

In [3]:
len(data)

1014

In [6]:
!python -m spacy init fill-config base_config.cfg config.cfg

2023-06-28 19:02:16.455548: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [5]:
%pip install spacy_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.8/190.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.4 MB/s eta 0:00:00


In [7]:
def get_spacy_doc(file,data):
  nlp = spacy.blank("en")
  db = DocBin()

  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    for start,end,label in annot:
      skip_entity = False
      for idx in range(start,end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity == True:
        continue

      entity_indices = entity_indices + list(range(start,end))
      try:
        span = doc.char_span(start,end,label=label,alignment_mode='strict')
      except:
        continue

      if span is None:
        err_data = str([start,end]) + "    "+ str(text) + "\n"
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db

In [8]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(cv_data,test_size=0.2)

In [9]:
file = open('train_file.txt','w')
db = get_spacy_doc(file,train)

100%|██████████| 811/811 [00:09<00:00, 84.56it/s] 


In [10]:
db.to_disk('train_data.spacy')

In [11]:
db = get_spacy_doc(file,test)

100%|██████████| 203/203 [00:02<00:00, 101.28it/s]


In [12]:
db.to_disk('test_data.spacy')

In [13]:
file.close()

In [14]:
!python -m spacy train config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

2023-06-28 19:11:40.061569: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-06-28 19:11:47,085] [INFO] Set up nlp object from config
[2023-06-28 19:11:47,101] [INFO] Pipeline: ['transformer', 'ner']
[2023-06-28 19:11:47,105] [INFO] Created vocabulary
[2023-06-28 19:11:47,105] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT 

In [ ]:
nlp = spacy.load('/output/model-best')

In [ ]:
!pip install PyMuPDF
import sys, fitz
fname = 'pdf'
doc = fitz.open(fname)

In [ ]:
text = " "
for page in doc:
  text = text+str(page.get_text())

In [ ]:
doc = nlp(text)
for ent in doc.ents:
  print(ent.text," ---->>>",ent.label_)